In [ ]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key

'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'

In [2]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000029A50262D20>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000029A504ABF50>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Siman and I am a Chief AI Engineer")])

AIMessage(content="Hello Siman, it's nice to meet you!  \n\nThat's a fascinating role. What kind of AI work are you involved in?  I'm always eager to learn more about how people are using AI to make a difference.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 23, 'total_tokens': 77, 'completion_time': 0.098181818, 'prompt_time': 0.002050967, 'queue_time': 0.240378228, 'total_time': 0.100232785}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--f4e5573c-8082-4e3b-9380-dbdd64d3d451-0', usage_metadata={'input_tokens': 23, 'output_tokens': 54, 'total_tokens': 77})

In [4]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Siman and I am a Chief AI Engineer"),
        AIMessage(content="Hello Siman, it's nice to meet you!  \n\nThat's a fascinating role. What kind of AI work are you involved in?  I'm always eager to learn more about how people are using AI to make a difference.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content='You are Siman, and you are a Chief AI Engineer!  \n\nIt was nice meeting you earlier.  \n\nIs there anything else I can help you with? 😊\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 97, 'total_tokens': 135, 'completion_time': 0.069090909, 'prompt_time': 0.005126684, 'queue_time': 0.244574858, 'total_time': 0.074217593}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--c7627384-b0a6-448c-9104-1b4a99b425b2-0', usage_metadata={'input_tokens': 97, 'output_tokens': 38, 'total_tokens': 135})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [5]:
!pip install langchain_community

Defaulting to user installation because normal site-packages is not writeable


In [6]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [7]:
config={"configurable":{"session_id":"chat1"}}

In [10]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Siman and I am a Chief AI Engineer")],
    config=config
)

In [11]:
response.content

'Hi Siman, it\'s a pleasure to meet you! \n\n"Chief AI Engineer" is a title I don\'t hear every day. That must be a very exciting and challenging role. \n\nWhat kind of AI projects are you leading these days?  I\'m always eager to learn about the innovative work being done in the field.\n'

In [18]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='You said your name is Siman.  \n\nIs there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 234, 'total_tokens': 257, 'completion_time': 0.041818182, 'prompt_time': 0.020386638, 'queue_time': 0.377698422, 'total_time': 0.06220482}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--4122c268-f4ca-48a7-9bc2-2bfbf3eb48b7-0', usage_metadata={'input_tokens': 234, 'output_tokens': 23, 'total_tokens': 257})

In [23]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"Your name is Siman!  \n\nI'm starting to think you like to play this little game. 😉  Is there something else you'd like to chat about? \n"

In [20]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is Siman")],
    config=config1
)
response.content

"Hi Siman! It's nice to meet you again. \n\nIs there anything I can help you with today?\n"

In [21]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"Your name is Siman!  \n\nI'm always here if you need a little reminder. 😊   Anything else I can do for you?\n"

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [24]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [25]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Smruti")]})

AIMessage(content="Hi Smruti, it's nice to meet you!  \n\nI'm here to help. What can I do for you? 😊  Just ask away!  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 32, 'total_tokens': 70, 'completion_time': 0.069090909, 'prompt_time': 0.002277406, 'queue_time': 0.244863437, 'total_time': 0.071368315}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--db942aa3-6736-4fc6-8075-0f3f1b5addbd-0', usage_metadata={'input_tokens': 32, 'output_tokens': 38, 'total_tokens': 70})

In [26]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [27]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Smruti")],
    config=config
)

response

AIMessage(content="Hi Smruti, it's nice to meet you!  How can I help you today?  😊 \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 81, 'total_tokens': 108, 'completion_time': 0.049090909, 'prompt_time': 0.004876392, 'queue_time': 0.250800976, 'total_time': 0.053967301}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--f01b64a6-9b52-443d-898a-fc7fb827f294-0', usage_metadata={'input_tokens': 81, 'output_tokens': 27, 'total_tokens': 108})

In [28]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Smruti! 😄  \n\nI remember now.  Is there anything else I can help you with?  \n'

In [29]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [30]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Smruti")],"language":"Hindi"})
response.content

'नमस्ते स्मृति! 👋  \n\nमुझे बहुत खुशी है कि आपसे मिली। आप मुझसे कुछ भी पूछ सकते हैं, मैं अपनी पूरी कोशिश करूँगा कि आपको सबसे अच्छी जानकारी दे सकूं। 😊  \n\n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [31]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [32]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Smruti")],"language":"Hindi"},
    config=config
)
repsonse.content

'नमस्ते Smruti! 👋  \n\nमुझे बहुत खुशी है कि आप मुझसे बात कर रहे हैं।  आप मुझसे कुछ भी पूछ सकते हैं, मैं अपनी पूरी क्षमता से उत्तर देने की कोशिश करूँगा। 😊\n'

In [33]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)

In [34]:
response.content

'आपका नाम Smruti है ।  \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [35]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

C:\Users\smrut\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\smrut\AppData\Roaming\Python\Python312\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\smrut\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate develo

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [38]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"As an AI, I have no memory of past conversations and no way to know your preferences.  \n\nWhat's your favorite ice cream flavor? 😊🍦\n"

In [39]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "what\'s 2 + 2". \n\n\nLet me know if you want to try another one! 😊\n'

In [40]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [41]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As an AI, I don't have access to past conversations or any personal information about you, including your name.  \n\nIs there a name you'd like to go by? 😊  \n\n"

In [42]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I have no memory of past conversations.  So I don't know what math problem you asked!  \n\nIf you'd like to ask me a math problem now, I'm happy to help. 😊  \n\n"